# Pandas assignment

The problems in this notebook are adapted from Brandon Rhodes's Pycon `pandas` tutorial.

The first few problems are identical.  Others are complete fabrications.

Problems that have an asterisk (`*`) are worth twice as  much.  They are more difficult.

Each of part A and Part B is handed in  as a separate assignment.  Each part is worth 20 points.
The point values for each question (out of a tital of 20) are as follows:

$$
\begin{array}{ll}
\text{Unstarred} & 1.428 \\
 \text{Starred} & 2.856\\
\end{array}
$$

## Loading the data

This notebook loads two datasets `titles` and ` cast`.  Both are are loaded frim the course website and contain extracts of the IMDB movie data. The second is a fragment of a much larger DataFrame.  

Everything
you need to complete this assignment is included in the **fragment loaded in this section**
in the cells below. 

For those who want to have the entire IMDB-derived dataset, here are some pointers.

Loading the entire IMDB data set used for these tutorial exercises is best done by 
visiting [Brandon Rhodes' github repo](https://github.com/brandon-rhodes/pycon-pandas-tutorial)
and following the instructions.  You can either download a big zip file or, if you have `git` (the main
github API function) installed on your machine, do a `git clone` command. 
In addition to that, 
you will need to download 4 compressed files by ftp that Rhodes
provides links to.  You can then run code
from Rhodes' cloned repo to create the non-truncated version CSV files used in these exercises.
If you do that, you may use the complete dataset to complete these exercises.  Some but not
all of your answers will be dufferent.

The statistics
in the Part B answers will not be correct until you load the complete dataset.  But
you can complete the assignment on the fragment, since I will be evaluating your code,
not the accuracy of your statistics.

#### Context Cell (Parameter-setting and Notebook-wide imports)

In [14]:
# The Python modules you need for this assignment.
import pandas as pd
import os.path
import urllib.request
import urllib.error
from matplotlib import pyplot as plt


pd.options.plotting.backend = 'matplotlib'


# Switch to True if loading the data below yields a 404 Not Found Error
#data_load_has_failed = False

github_url ='https://raw.githubusercontent.com/gawron/python-for-social-science/master/'
data_path = 'pandas/tutorial/pycon-pandas-tutorial-master/truncated_data/'
titles_file = 'titles.csv'
cast_file = 'cast.csv'
truncated_cast_file = 'truncated_cast.csv'
url_dir = github_url + data_path
#url_dir = 'https://gawron.sdsu.edu/python_for_ss/course_core/data/'

In [15]:
# Data is on github.  Must be connected to the internet to get the data.
url_dir

'https://raw.githubusercontent.com/gawron/python-for-social-science/master/pandas/tutorial/pycon-pandas-tutorial-master/truncated_data/'

The next cell adds styles to the notebook. Helps make the assignment questions easier to find.

Please execute:

In [3]:
from IPython.display import HTML
import urllib
import os.path

style_dir = 'https://gawron.sdsu.edu/python_for_ss/course_core/data/'
target_url1 = os.path.join(style_dir,'style-notebook.css')
target_url2 = os.path.join(style_dir,'style-table.css')

# There are issues with doing it this way on Windows
#target_url1 = style_dir + '/style-notebook.css'
#target_url2 = style_dir + '/style-table.css'

with urllib.request.urlopen(target_url1) as fh1:
    css1 = fh1.read().decode('utf8')
with urllib.request.urlopen(target_url2) as fh2:
    css2 = fh2.read().decode('utf8')
css = css1 + css2
# This attaches the style sheet to the current notebook
HTML(f'<style>\n{css}</style>')
# print("Hi")
# Note that the HTML function-call above returns an uprintable html-object.
# It must be the last expression executed in the cell in an unembedded block of code.
# It does not have the intended effect on the styling of  the notebook
# if the print statement above is un-(commented out)
# or if it is embedded in an if-statement as below.  It must be the value 
# returned by the cell (what is printed out, if printable, )
# if True:
#    HTML(f'<style>\n{css}</style>')

The next cell loads the `titles` DataFrame, the first of two used in this assignment.  There are only
two columns, `'title'`  and `'year'`.

In [16]:
titles = pd.read_csv(url_dir + titles_file)
titles.head()

,title,year
0,The Patriarchs,2009
1,Angels in the Attic,1998
2,The Rapture,1991
3,Star na si Van Damme Stallone,2016
4,Sweet Talk,2004


This is a simple `DataFrame` with two columns, containing the title and year of a film.

If a film is remade and given the same title, the title shows up twice:

In [17]:
titles[titles['title']  == 'Around the World in 80 Days']

,title,year
91875,Around the World in 80 Days,2004
121800,Around the World in 80 Days,1956


In [18]:
len(titles)

226013

The next cell loads the `cast` DataFrame, the second of two used in this assignment.  This is
large and will take a while.

In [19]:
#full_cast = pd.read_csv('tutorial/pycon-pandas-tutorial-master/data/cast.csv')
#cast = pd.read_csv('tutorial/pycon-pandas-tutorial-master/data/truncated_cast.csv',
#                   index_col=0)
cast = pd.read_csv(url_dir + truncated_cast_file)

In [20]:
len(cast)

423121

In [155]:
cast.head()

,title,year,name,type,character,n
0,In the Land of the Head Hunters,1914,Paddy 'Malid,actor,Kenada,5.0
1,The Colour of Darkness,2016,Ketan Daraji -Gohel,actor,Chhagan,NaN
2,London Betty,2009,Isaiah Entsua -Mensah,actor,Camera Man,NaN
3,Candelabra,2014,Groovin .,actor,Lt. Dick Sims,5.0
4,Bad Ideas,2012,Hamid .,actor,The Diner,NaN


In [6]:
cast["title"]

0         In the Land of the Head Hunters
1                  The Colour of Darkness
2                            London Betty
3                              Candelabra
4                               Bad Ideas
                       ...               
422955                    Two Way Stretch
422956                       Up the Creek
422957             Waltz of the Toreadors
422958                What's New Pussycat
422959                Where Does It Hurt?
Name: title, Length: 422960, dtype: object

Please have a look at the columns of the `cast` `DataFrame`and make sure you understand what information it contains. A row uniquely identifies a role in a film; it contains the film title, the year the file was made, the name of the actor or actress playing the role, and the name of the character they played.  The number in the `'n'` column represents the importance of the part, with the lead role receiving a 1 and all less important roles receiving higher numbers. 

Sometimes extras are included; with extras included, a big Hollywood production can easily  cast over 1000 actors and will therefore populate over 1000 rows of `cast`.  The character name for the extras will be "Extra".

In [21]:
len(cast[(cast['title']  == 'Around the World in 80 Days')&(cast['year']  == 1956)])

1299

In [22]:
cast[(cast['title']  == 'Around the World in 80 Days')&(cast['year']  == 1956)].head()

,title,year,name,type,character,n
784,Around the World in 80 Days,1956,Ronald Adam,actor,Club Steward,47.0
7147,Around the World in 80 Days,1956,Ray Arnett,actor,Extra,NaN
25665,Around the World in 80 Days,1956,Charles Boyer,actor,Monsieur Gasse - Thomas Cook Paris Clerk,5.0
40928,Around the World in 80 Days,1956,Suey Chan,actor,Extra,NaN
44164,Around the World in 80 Days,1956,Martin Cichy,actor,Bartender,NaN


As shown in the next cell, there are a number of null or `Nan` values in the `'n'` column,
roles for which no ``n`-value is available.

In [23]:
len(cast[cast['n'].isnull()])

154710

There are also some very
high numbers in the `'n'` column, but not very many of them.

In [24]:
len(cast[cast['n']>500])

9

Note:  A single actor may play more than one role in a film.  In the example below, both roles are the starring role.  I guess Jackie Chan doesn't take a back seat to anyone.

In [206]:
cast[(cast['title']=='Around the World in 80 Days')&(cast['year']==2004)&(cast['n']==1)]

,title,year,name,type,character,n
351948,Around the World in 80 Days,2004,Jackie Chan,actor,Passepartout,1.0
351949,Around the World in 80 Days,2004,Jackie Chan,actor,Lau Xing,1.0


In [ ]:
pd.crosstab(cast.name,cast.character).sum(axis=1)

In [56]:
cast.groupby("name")["character"].value_counts().sort_values(ascending=False)[:10]

name                 character
Herman Hack          Henchman     58
Tex Palmer           Henchman     57
George Morrell       Townsman     46
Blackjack Ward       Henchman     45
Jim Corey            Henchman     44
Horace B. Carpenter  Townsman     42
Tarzan               Tarzan       42
Art Dillard          Henchman     41
Al Taylor            Henchman     38
Don Brodie           Reporter     38
Name: count, dtype: int64

In [50]:
pd.pivot_table(cast,index= ("name","character"),
               aggfunc="count")["type"].sort_values(ascending=False)

name             character             
Herman Hack      Henchman                  58
Tex Palmer       Henchman                  57
George Morrell   Townsman                  46
Blackjack Ward   Henchman                  45
Jim Corey        Henchman                  44
                                           ..
Gay Goodwin      Waitress                   1
Gay DeLys        Chorine                    1
Gay Christie     Mary Steinweg, Tochter     1
Gavril Vitanov   Conductor                  1
Þór Jóhannesson  Police Detective 2         1
Name: type, Length: 404393, dtype: int64

19.991999999999997

## Part A Questions involving selecting and sorting subsets of the rows in the Dataset

### A.1 How many movies are listed in the titles dataframe?

As an example, we have put the answer in the next cell.  Be sure to execute the cell containing your solution, so that the answer to the question is displayed in the output as it is in this example.

For each question, first give some thought to which of the two DataFrames loaded in the section entitled "Loading the Data" is best suited to provide an answer.

In [14]:
len(titles)

226013

### A.2 How many movies have the title "Hamlet"?

Hint: one approach is to use a Boolean mask

### A.3  When was the first movie titled "Hamlet" made?

### A.4 How many movies were made from 1950 through 1959?

### A.5 In what years has a movie titled "Batman" been released?

###  A.6 How many credited roles are there in the movie "Inception"?  We'll interpret  this to mean we should not count roles that have no `n`-value.

The idea is that some of the rows have `NaN` in the `'n'` column.  These are not
values.  To eliminate such rows use the `.notnull()` method.

###  A.7 * How many roles were credited in the version of Hamlet in which Kenneth Branagh plays Hamlet?


### A.8  How many roles for a character named "Hamlet" have there been in films not entitled "Hamlet"?  

###  A.8  * How many people have played "James Bond" as a leading role?

Yes, I'm thinking of the Ian Fleming character that has given rise to a whole film franchise,
but I'm going to allow for a little noise.  It
turns out that characters named "James Bond" have come up many times in film history.  Adding
the qualification "as a leading role" helps with that.
This will still leave in a couple of non-Ian Fleming Bonds, but don't worry.
If you just answer the question literally, your
answer should be very close to a list of the actors who've played the
Ian Fleming character.  By the way,  David Niven will be missing from
that list, because the name of the character
listed in the credits of his 1967 parody is not "James Bond" but "Sir James Bond".
It's okay to leave out David Niven.  It was a parody, not a real Bond film.

Note: Relative to this DB (which is dated), the historically correct answer to the question how many actors played the Ian Fleming character is 6. The literal answer to the question in this DB is 
greater than 6 because of a few superfluous Bonds, but if your answer is a lot more than 6
(greater than 10), then there's an issue with your code.  There is a technical challenge in this question;
it is tricky to get exactly the right list of actors.  Pay attention to the English.  It's how many peoples, not how many roles.

###  A.9  *  List the films and the characters in which Paul Newman played a role he had played before.

Hint: The `.duplicated()`  method may be of help. 

Sad note. If you are any kind of Paul Newman fan you will see there is a
very important role being left out if you compute this answer in a logical
way.  This is because the name of the character
has been slightly altered in the second movie.  Don't try to fix this.  Let's
just call this data corruption.

###  A.10  * List all characters whose name contains the string "Butterfly" 

Ignore case.  Your answer should include characters whose name is "Butterfly"
or "Monarch the butterfly", as well as those whose last name is "Butterfly" (such as "Madame Butterfly"), as well as those whose last name is "Butterfield" or "Butterworth", and so on.

Note:  Try using  a string method on the appropriate column using `.str`.

## Part B:  Questions needing value_counts, pivot_tables, or cross tabulation

There are two series of questions in Part B, questions involving the cast and title
data used in Part A, and questions involing a new data set you will load below.   Completing Part B involves answering both series of questions.

Brief discussion of an important issue for the data below.

Some of the numbers in the following table are going
to be misleading:

In [19]:
pt000 = cast.pivot_table(index='title',aggfunc='size').sort_values(ascending=False)

It looks as if this might give us film cast sizes (`.size()` tells how many rows in each group, and rows are roles).

In [24]:
pt000[:5]

title
Around the World in 80 Days     1364
Hamlet                           342
Mr. Smith Goes to Washington     217
Wells Fargo                      193
Union Pacific                    183
dtype: int64

But that's not quite right: And here's why:

In [25]:
len(cast[cast['title']=='Around the World in 80 Days'])

1364

In [23]:
cast[cast['title']=='Around the World in 80 Days']['year']

784       1956
7147      1956
25665     1956
40851     2004
40852     2004
          ... 
422807    1956
422808    1956
422809    2004
422810    1956
422811    1956
Name: year, Length: 1364, dtype: int64

So 1364 is actually the size of the combined cast of two films, one made in 1956 and the other in 2004.  Not whjat we thought we were getting.  Let's call this the **remake** issue.  Bear it in mind as you answer the questions below.

### Discussion question: Of the films made before 1939, which had the largest cast?

After you've made a DataFrame containing the set of rows in `cast` that you're interested in,
you need to do a computation involving counting.  The question to think about
is what you want to count.  You **don't** want to do this:

```
pd.crosstab(cast_pre_1939['title'],cast_pre_1939['name'])
```

since after a **very** long computation, this will end up telling you
the number of characters each actor/actress played in each film
(which is usually 1).

So think about the structure of the data.  This is actually pretty simple.


Films made before 1939:

In [60]:
cast_pre_1939 = cast[cast['year'] < 1939]

The simplest answer is **wrong**, because of the remake issue.

In [61]:
title_counts_pre_1939 = cast_pre_1939['title'].value_counts()
title_counts_pre_1939.head()

title
Wells Fargo                   193
Mr. Deeds Goes to Town        177
The Buccaneer                 172
The Merry Widow               164
You Can't Take It with You    161
Name: count, dtype: int64

Consider:

In [62]:
title_counts_pre_1939["Hamlet"]

37

It's wrong because a film title like  "Hamlet" has had  many remakes and
`title_counts_pre_1939` conflates these films:

In [63]:
cast_pre_1939[cast_pre_1939['title'] == 'Hamlet']['year'].unique()

array([1921, 1913, 1911, 1910])

Any solution that groups by year and title will avoid this probem.  For example:

In [64]:
cgp1939= cast_pre_1939.groupby(['title','year'])['character'].count()
cgp1939.loc["Hamlet"]

year
1910     1
1911     6
1913    21
1921     9
Name: character, dtype: int64

These are the correct cast counts for versiuonbs of "Hamlet" made before 1939:

### B.1 Of the films made in either 1939 or 1966, what films had a cast size of 90 or more but fewer than 100?


After you've made a DataFrame containing the set of rows in `cast` that you're interested in,
you again need to do a computation that's counts cast members on films.   There is a simple computation  that seems to give you the cast sizes of all films.

But to answer this question correctly,  you will need to
come up with a slightly more complicated answer. The issue to
think about is remakes.  In fact in 1966 there were some remakes of
great 1939 films.  How should that affect your answer to this
problem?


###  B.2 * What character has been played the most times by a single actor in a starring role?

### B.3 Produce a pie  chart showing what percentages of all roles go to actors and actresses respectively (so, a two color pie chart)

Note:  This can be done with a `.plot()` method called on a pandas `DataFrame` or `Series`.  See if you can figure out how to make that happen.   No pivot table or any grouping operation required.

### B.4 * Plot the percentages of all roles that are female year by year for the century from 1917 through 2017

You are continuing your study of the imbalance of male and female roles.  Years on the x-axis, one line tracking the percentage of female roles.

This is Time Series data. Do a second plot that smooths this jiggly plot and **highlights the trend** by plotting the rolling  mean of the percentage of actress roles, using a 10 year window. Hint: Look at the `.rolling()` method.   Or even better, for extra glory, get both plots in the same subplot (the matplotlib term is "the same axis"). So you will have one pair of x,y axes and two lines in the plot, one a smoother version of the other.

###   B.5  * Among actresses who have roles in more than 3 decades, what actress has the largest average number of roles per decade?  What is that average number of roles?


Here is an important wrinkle in computing averages.  We have data for 14 decades (1890s through 2020s).  For each actress, compute her average roles per decade by averaging only over decades in which she had roles. Consider, for example, the actress Grace Hayle, who had 104 roles in the 1930s and no roles in any other decade. Her average roles per decade should be 104, not 104/14.  Of course that average isn't a candidate for the answer to this question, since she didn't have roles in more than 3 decades.




### C.1 A new dataset.  Load the word frequency data from the following URL.


```python
"https://www.kilgarriff.co.uk/BNClists/lemma.num"
```

This is frequency data compiled from the **British National Corpus**
a balanced data set of English texts of
about 100 million words. You should use the pandas function `read_csv` to read the
tabular data from the above URL.  The DataFrame should have the following format,

```

   Word     Rank    Freq	 Pos
			
 0  the	      1	   6187267	det
 1  be	      2	   4239632	v
...
```

In other words, there should be four columns named "Word", "Rank", Freq" and "Pos"
(the order doesn't matter).  The display above shows the first two rows.  Make sure you
don't lose any rows of data. This will take a small bit of tweaking of the
arguments of `read_csv`.

The counts in this data are **lemma** counts. The lemma of a word is
the form used in its dictionary entry.  When we refer to
lemma properties we are referring to properties that apply 
equally to all forms of a word, such as its meaning.   What that means for this
data set is that the counts in the
`"Freq"` column combine the counts of all forms of the word in the `"Word"` column.
For example, the count for *be* in the second row combines
the counts for all forms the verb *be*, including *is*, *are*, *was*, *were*, and
*being*.


**Answer the following question**.  Does the index provide a complete vocabulary list for
all the  words in the BNC corpus, or has it been truncated in some way?  Why
or why not?

**Important note**:  If you have trouble getting this data loaded in the proper format, please ask for
help.  The answers to all the following questions depend on getting this one right.  And you can;t even get partial credit for nthose questions without getting this one right.

### C.2 Use the word frequency data just loaded to get the total word counts by part of speech (Pos)

Note:  the numbers should mostly be in the millions.

**Answer the following question**:  What part of speech occurs  the least number of times?

### C.3 Using the word frequency data again, plot the rank versus the frequency

Demonstrate something called **Zipf's Law** by using a log scale for both
your x- and your y- axes.  Note:   This plot is pretty uninteresting
if you don't do this.

### C.4 * Using the BNC word frequency data again, find the word with the most parts of speech.  What are those parts of speech?

### C.5  Using the BNC word frequency data again, find all words that can be both verbs and nouns and count them.  What percentage of nouns can also be verbs?